In [1]:
import numpy
import pandas as pd
import matplotlib as plt
import numpy as np


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import LabelEncoder,RobustScaler
from sklearn.model_selection import KFold, cross_val_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
newTrain=train[['SalePrice','LotArea','OverallQual','OverallCond','YearBuilt']]
pd.options.mode.chained_assignment = None

In [3]:
#Using log transformation on variable we are predicting to get rid of skewness
newTrain['SalePrice']=np.log1p(newTrain['SalePrice'])

In [4]:
X_test=test[['LotArea','OverallQual','OverallCond','YearBuilt']]

In [5]:
X_train=newTrain[[c for c  in newTrain.columns if c not in ['SalePrice']]].values
log_y_train=newTrain['SalePrice'].ravel()


model=LinearRegression()

In [6]:
model.fit(X_train,log_y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
log_y_pred=model.predict(X_train)

In [8]:
log_y_test=model.predict(X_test)

In [9]:
y_pred=np.expm1(log_y_pred)
y_test=np.expm1(log_y_test)
y_train=np.expm1(log_y_train)


In [10]:
# Calculate metrics
print("Mean Absolute Error: %.2f" %mean_absolute_error(y_train, y_pred))
print("Mean Squared Error: %.2f" %mean_squared_error(y_train, y_pred))
print("Root Mean Squared Error: %.2f" %np.sqrt(mean_squared_error(y_train, y_pred)))
print("Coefficient of Determination: %.2f" %r2_score(y_train, y_pred))

Mean Absolute Error: 28138.82
Mean Squared Error: 2051958988.96
Root Mean Squared Error: 45298.55
Coefficient of Determination: 0.67


In [11]:

subDf=pd.DataFrame({'Id':test['Id'].values})
subDf['SalePrice']=y_test
subDf.to_csv("submission.csv", index=False)